In [1]:
import os
from pathlib import Path

In [2]:
DATA_ROOT_PATH = Path('/data/k_water/dataset')

TRAIN_DATA_IMAGE = sorted(DATA_ROOT_PATH.rglob("train/train_*.png"))
TEST_DATA_IMAGE = sorted(DATA_ROOT_PATH.rglob("test/test_*.png"))
TRAIN_DATA_ANNOT = DATA_ROOT_PATH / "labels" / "train.json"

ANSWER_SAMPLE = DATA_ROOT_PATH / "labels" / "answer_sample.json"

In [6]:
print(len(TRAIN_DATA_IMAGE))
print(len(TEST_DATA_IMAGE))

104875
44946


In [7]:
TRAIN_DATA_ANNOT

PosixPath('/data/k_water/dataset/labels/train.json')